# 🚀 Autorender AI - Colab Quickstart

In [ ]:
# 🚀 AutoRender AI - Google Colab Quickstart

This notebook sets up the **AutoRender AI microservice** on Google Colab with:
- 🎨 Background removal with edge refinement
- 🌟 AI background generation using Stable Diffusion  
- 🔍 Object detection and smart cropping
- 👤 Face detection and cropping
- ⚡ GPU acceleration
- 🌐 Public URL via ngrok tunnel

**⚠️ Important:** Make sure to enable GPU runtime: `Runtime > Change runtime type > Hardware accelerator > GPU`

---

## 📋 Step 1: Environment Setup & GPU Check

In [ ]:
import torch
import sys
import os

print("🖥️  System Information:")
print(f"   Python: {sys.version.split()[0]}")
print(f"   PyTorch: {torch.__version__}")
print(f"   CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"   CUDA Version: {torch.version.cuda}")
else:
    print("   ⚠️  No GPU detected. Enable GPU in Runtime > Change runtime type")

print("\n" + "="*50)

In [ ]:
## 📦 Step 2: Clone Repository & Install Dependencies

In [ ]:
# Clone the AutoRender AI repository
print("📥 Cloning AutoRender AI repository...")
!git clone https://github.com/BalajiAIDev/autorender-ai.git
%cd autorender-ai

print("✅ Repository cloned successfully!")
print("\n📦 Installing system dependencies...")

# Install system dependencies
!apt-get update -qq
!apt-get install -y -qq libgl1-mesa-glx libglib2.0-0 libsm6 libxext6 libxrender-dev libgomp1

print("✅ System dependencies installed!")

In [ ]:
print("📦 Installing Python dependencies...")

# Install PyTorch with CUDA support first
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install other dependencies from requirements.txt
!pip install -r requirements.txt

print("✅ All Python dependencies installed!")

# Verify key imports
print("\n🔍 Verifying key imports...")
try:
    import flask
    import rembg
    import ultralytics
    import diffusers
    import pyngrok
    print("✅ All critical packages imported successfully!")
except ImportError as e:
    print(f"❌ Import error: {e}")

In [ ]:
## 🔑 Step 3: Setup Ngrok (Optional - for permanent URLs)

In [ ]:
# Optional: Add your ngrok auth token for permanent URLs
# Get your token from: https://dashboard.ngrok.com/get-started/your-authtoken

NGROK_AUTH_TOKEN = ""  # Add your token here if you have one

if NGROK_AUTH_TOKEN:
    from pyngrok import ngrok
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("🔑 Ngrok auth token configured!")
else:
    print("ℹ️  No ngrok auth token provided. You'll get a temporary URL.")
    print("   To get a permanent URL, sign up at https://ngrok.com")

# Set environment for Colab
import os
os.environ['FLASK_ENV'] = 'colab'
os.environ['ENABLE_NGROK'] = 'true'

In [ ]:
## 🚀 Step 4: Start AutoRender AI Service

In [ ]:
import threading
import time
import subprocess

print("🚀 Starting AutoRender AI service...")
print("⏳ This will take a moment to load AI models...")

# Start the Flask app with ngrok in background
def start_app():
    subprocess.run(["python", "app.py"], capture_output=False)

# Start server in background thread
server_thread = threading.Thread(target=start_app)
server_thread.daemon = True
server_thread.start()

print("✅ Server starting... Please wait for ngrok URL to appear above.")
print("🔍 Look for a line that shows: 'running on https://xxxxx.ngrok.io'")

# Wait for server to start
time.sleep(10)

In [ ]:
## 🧪 Step 5: Test the Service

In [ ]:
import requests
import json
import time

# Wait a bit more for full startup
time.sleep(5)

print("🔍 Testing the AutoRender AI service...")

# Test local health endpoint first
try:
    response = requests.get("http://localhost:5000/health", timeout=10)
    if response.status_code == 200:
        print("✅ Local health check passed!")
        print(json.dumps(response.json(), indent=2))
    else:
        print(f"❌ Local health check failed: {response.status_code}")
except Exception as e:
    print(f"⚠️  Local connection issue: {e}")
    print("Server might still be starting up...")

print("\n📋 Service Status:")
try:
    response = requests.get("http://localhost:5000/status", timeout=10)
    if response.status_code == 200:
        status = response.json()
        print(f"✅ Service running with {len(status.get('endpoints', {}))} endpoint categories")
        print(f"🤖 Models available: {list(status.get('models', {}).keys())}")
    else:
        print(f"❌ Status check failed: {response.status_code}")
except Exception as e:
    print(f"⚠️  Status check issue: {e}")

In [ ]:
## 🎨 Step 6: Test Background Removal

In [ ]:
import requests
import base64
from PIL import Image, ImageDraw
from io import BytesIO
import matplotlib.pyplot as plt

print("📷 Creating test image for background removal...")

# Create a simple test image
width, height = 400, 400
image = Image.new('RGB', (width, height), 'white')
draw = ImageDraw.Draw(image)

# Draw a red circle
center_x, center_y = width // 2, height // 2
radius = 80
draw.ellipse(
    [center_x - radius, center_y - radius, center_x + radius, center_y + radius],
    fill='red',
    outline='darkred',
    width=3
)

# Add some text
try:
    draw.text((center_x - 30, center_y - 10), "TEST", fill='white')
except:
    draw.text((center_x - 20, center_y - 5), "AI", fill='white')

# Save test image
image.save('test_image.jpg', 'JPEG')

# Display the test image
plt.figure(figsize=(6, 6))
plt.imshow(image)
plt.title("Original Test Image")
plt.axis('off')
plt.show()

print("✅ Test image created successfully!")

In [ ]:
print("🎨 Testing background removal API...")

try:
    # Test background removal with green background
    with open('test_image.jpg', 'rb') as f:
        files = {'image': f}
        data = {'bg_color': '#00ff00'}  # Green background
        
        print("📤 Sending request to /remove-bg...")
        response = requests.post(
            "http://localhost:5000/remove-bg", 
            files=files, 
            data=data,
            timeout=60  # Give it time for model loading
        )
    
    if response.status_code == 200:
        result = response.json()
        if result.get('success'):
            print("✅ Background removal successful!")
            
            # Decode and display the result
            image_data = base64.b64decode(result['image'])
            result_image = Image.open(BytesIO(image_data))
            
            # Display original and processed images side by side
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
            
            ax1.imshow(image)
            ax1.set_title("Original Image")
            ax1.axis('off')
            
            ax2.imshow(result_image)
            ax2.set_title("Background Removed (Green BG)")
            ax2.axis('off')
            
            plt.tight_layout()
            plt.show()
            
            print("🎉 Background removal test completed successfully!")
            print("💡 You can now use the public ngrok URL to access the API from anywhere!")
        else:
            print(f"❌ API returned error: {result}")
    else:
        print(f"❌ Request failed with status {response.status_code}")
        print(f"Response: {response.text[:500]}")
        
except Exception as e:
    print(f"❌ Test failed: {e}")
    print("💡 This is normal if models are still loading. Try running this cell again in a minute.")

In [ ]:
## 📝 Step 7: Get Your Public URL & API Usage

Your AutoRender AI service is now running! 🎉

In [ ]:
# Get the ngrok public URL
try:
    from pyngrok import ngrok
    tunnels = ngrok.get_tunnels()
    if tunnels:
        public_url = tunnels[0].public_url
        print(f"🌐 Your PUBLIC AutoRender AI URL: {public_url}")
        print("=" * 60)
        print()
        print("📋 Available API Endpoints:")
        print(f"   • {public_url}/health - Health check")
        print(f"   • {public_url}/status - Service status & model info")
        print(f"   • {public_url}/remove-bg - Background removal")
        print()
        print("💡 Usage Examples:")
        print(f"""
🔹 Test with curl:
   curl -X POST -F 'image=@your_image.jpg' {public_url}/remove-bg

🔹 With background color:
   curl -X POST -F 'image=@your_image.jpg' -F 'bg_color=#ff0000' {public_url}/remove-bg

🔹 With edge blur:
   curl -X POST -F 'image=@your_image.jpg' -F 'edge_blur_radius=2' {public_url}/remove-bg

🔹 Test from Python:
   import requests
   with open('image.jpg', 'rb') as f:
       response = requests.post('{public_url}/remove-bg', 
                               files={{'image': f}},
                               data={{'bg_color': '#ffffff'}})
   result = response.json()
        """)
        
        print("\n🔐 Save this URL - it will work until you stop this notebook!")
        
    else:
        print("⚠️  No ngrok tunnels found.")
        print("🌐 Service is running locally on: http://localhost:5000")
        
except Exception as e:
    print("ℹ️  Could not retrieve ngrok URL")
    print("🌐 Service is running locally on: http://localhost:5000")

print("\n" + "=" * 60)
print("🎉 AutoRender AI is ready to use!")
print("🔥 The server will keep running until you stop this notebook.")
print("📱 You can test it from any device using the public URL above.")

In [ ]:
## 🔧 Troubleshooting

If you encounter any issues:

1. **Models loading slowly**: This is normal on first run. Wait 2-3 minutes.
2. **GPU not detected**: Go to `Runtime > Change runtime type > Hardware accelerator > GPU`
3. **Ngrok errors**: The free version has limitations. Try restarting the notebook.
4. **API errors**: Check the server logs above for detailed error messages.

## 🎯 Next Steps

- Upload your own images to test
- Try different background colors
- Integrate with your applications
- Check the [GitHub repository](https://github.com/BalajiAIDev/autorender-ai.git) for more features

**Happy AI processing! 🚀**